# Environement Setup

In [1]:
# @title 1. 🛠️ Installazione Ambiente (PyTorch 2.4.0)
import os
import sys

print("🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)")

# 1. Disinstallazione versioni incompatibili
os.system("pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric")

# 2. Installazione PyTorch 2.4.0 (Versione richiesta dal README)
# Usiamo CUDA 12.1 che è stabile su Colab
cmd_torch = "pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121"
os.system(cmd_torch)

# 3. Installazione librerie Grafiche (PyG) specifiche per Torch 2.4.0
pyg_url = "https://data.pyg.org/whl/torch-2.4.0+cu121.html"
libs = ["torch-scatter", "torch-sparse", "torch-cluster", "torch-spline-conv", "torch-kmeans"]
cmd_pyg = f"pip install {' '.join(libs)} -f {pyg_url}"
os.system(cmd_pyg)

# 4. Altre dipendenze
os.system("pip install torch-geometric hydra-core omegaconf submitit pandas ftfy")

os.system("pip install -r requirements.txt")

print("✅ Installazione completata.")
print("⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!")

🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)
✅ Installazione completata.
⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!


In [1]:
# @title 2. 📂 Setup HiERO e Download Pesi Pre-addestrati
import os
from google.colab import userdata

# 1. Recupero Token e Clone del repository Privato
repo_name = "HiERO-main"
if not os.path.exists(repo_name):
    print(f"⬇️ Clonazione repository {repo_name} (Privato)...")
    try:
        # Recupera il token dai segreti di Colab
        token = userdata.get('GITHUB_TOKEN')

        # Costruisce l'URL autenticato: https://TOKEN@github.com/...
        repo_url = f"https://{token}@github.com/MarcoPernoVDP/HiERO-main.git"

        # Esegue il clone (l'output viene nascosto per sicurezza se usi capture_output,
        # ma os.system standard è ok, basta non stampare l'URL completo)
        ret = os.system(f"git clone --recursive {repo_url}")

        if ret != 0:
            print("❌ Errore durante il clone. Verifica che il token sia corretto e abbia i permessi.")
    except Exception as e:
        print(f"❌ Errore nel recupero del token: {e}")
        print("Assicurati di aver aggiunto 'GITHUB_TOKEN' nei Segreti di Colab (icona chiave a sinistra).")
else:
    print(f"✅ Repository {repo_name} già presente.")

# Cambiamo directory se il clone ha avuto successo o la cartella esisteva
if os.path.exists(f"/content/{repo_name}"):
    os.chdir(f"/content/{repo_name}")
    print(f"📂 Directory di lavoro: {os.getcwd()}")

    # 2. Creazione struttura cartelle
    dirs = [
        "checkpoints",
        "pretrained",
        "data/ego4d/raw/annotations",
        "data/ego4d/raw/features/egovlp"
    ]
    for d in dirs:
        os.makedirs(d, exist_ok=True)

    # 3. Download del modello HiERO già addestrato (Backbone EgoVLP)
    if not os.path.exists("checkpoints/hiero_egovlp.pth"):
        print("⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...")
        os.system("wget -O checkpoints/hiero_egovlp.pth https://www.sapeirone.it/data/hiero/ckpt/hiero_egovlp.pth")

    # 4. Download componenti testuali (richiesti dal codice)
    print("⬇️ Scaricamento Encoder Testuale...")
    os.system("wget -q -O pretrained/egovlp_text.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_text.pth")
    os.system("wget -q -O pretrained/egovlp_txt_proj.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_txt_proj.pth")

    # 5. Download Annotazioni necessarie (EgoMCQ/EgoClip)
    print("⬇️ Scaricamento Annotazioni...")
    os.system("gdown --id 1-aaDu_Gi-Y2sQI_2rsI2D1zvQBJnHpXl -O data/ego4d/raw/annotations/egoclip.csv")
    os.system("gdown --id 1-5iRYf4BCHmj4MYQYFRMY4bhsWJUN3rW -O data/ego4d/raw/annotations/egomcq.json")

    print("✅ Setup completato. Cartelle pronte.")
else:
    print("⚠️ Impossibile procedere: la cartella del repository non esiste.")

⬇️ Clonazione repository HiERO-main (Privato)...
📂 Directory di lavoro: /content/HiERO-main
⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...
⬇️ Scaricamento Encoder Testuale...
⬇️ Scaricamento Annotazioni...
✅ Setup completato. Cartelle pronte.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import zipfile

zip_path = "/content/drive/MyDrive/MistakeDetection/egovlp.zip"
extract_path = "/content/data/egovlp"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive


In [4]:
%cd /content/HiERO-main

!python batch_infer_text.py \
  --recipes-dir /content/drive/MyDrive/MistakeDetection/task_graphs \
  --checkpoint /content/HiERO-main/checkpoints/hiero_egovlp.pth \
  --output-dir /content/outputs/recipe_embeddings \
  --format pt \
  --device cuda

/content
🍳 Trovati 24 file JSON da processare
📁 Output directory: /content/outputs/recipe_embeddings
💾 Formato: pt
🖥️  Device: cuda

[1/24] 🔄 Processing: herbomeletwithfriedtomatoes
✅ Completato: herbomeletwithfriedtomatoes

[2/24] 🔄 Processing: mugcake
✅ Completato: mugcake

[3/24] 🔄 Processing: capresebruschetta
✅ Completato: capresebruschetta

[4/24] 🔄 Processing: spicytunaavocadowraps
✅ Completato: spicytunaavocadowraps

[5/24] 🔄 Processing: dressedupmeatballs
✅ Completato: dressedupmeatballs

[6/24] 🔄 Processing: breakfastburritos
✅ Completato: breakfastburritos

[7/24] 🔄 Processing: cucumberraita
✅ Completato: cucumberraita

[8/24] 🔄 Processing: microwaveeggsandwich
✅ Completato: microwaveeggsandwich

[9/24] 🔄 Processing: panfriedtofu
✅ Completato: panfriedtofu

[10/24] 🔄 Processing: broccolistirfry
✅ Completato: broccolistirfry

[11/24] 🔄 Processing: sautedmushrooms
✅ Completato: sautedmushrooms

[12/24] 🔄 Processing: microwavemugpizza
✅ Completato: microwavemugpizza

[13/24] 🔄 

In [5]:
import torch

data = torch.load('/content/outputs/recipe_embeddings/herbomeletwithfriedtomatoes.pt', weights_only=False)
embeddings = data['embeddings']
metadata = data['metadata']

In [8]:
embeddings

tensor([[-0.0188, -0.0186,  0.0290,  ...,  0.0357, -0.0007,  0.0550],
        [ 0.0170, -0.0333, -0.0212,  ..., -0.0229, -0.0482,  0.0337],
        [-0.0165, -0.0541,  0.0409,  ...,  0.0159, -0.0110,  0.0698],
        ...,
        [ 0.0462,  0.0636,  0.0060,  ..., -0.0319,  0.0183,  0.0667],
        [ 0.0126, -0.0095,  0.0395,  ..., -0.0325, -0.0647,  0.0300],
        [ 0.0319,  0.0140,  0.0795,  ..., -0.0156,  0.0229,  0.0403]])

In [7]:
metadata

{'edges': [[14, 1],
  [12, 1],
  [10, 2],
  [7, 4],
  [1, 6],
  [2, 7],
  [3, 8],
  [11, 9],
  [6, 10],
  [4, 11],
  [13, 12],
  [8, 14],
  [5, 14],
  [15, 14],
  [0, 3],
  [0, 5],
  [0, 13],
  [0, 15],
  [9, 16]],
 'step_mapping': {'1': 0,
  '2': 1,
  '3': 2,
  '4': 3,
  '5': 4,
  '6': 5,
  '7': 6,
  '8': 7,
  '9': 8,
  '10': 9,
  '11': 10,
  '12': 11,
  '13': 12,
  '14': 13,
  '15': 14},
 'step_ids': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15'],
 'original_steps': {'0': 'START',
  '1': 'Heat-Heat 1 tbsp oil in a non-stick frying pan',
  '2': 'put-put tomatoes on a serving plate',
  '3': 'Chop-Chop 2 tbsp cilantro',
  '4': 'stir-stir gently with a wooden spoon so the egg that sets on the base of the pan moves to enable the uncooked egg to flow into the space',
  '5': 'crack-crack one egg in a bowl',
  '6': 'cook-cook the tomatoes cut-side down until they start to soften and colour',
  '7': 'Pour-Pour the egg mixture into

In [9]:
!cp -r /content/outputs/recipe_embeddings/ /content/drive/MyDrive/MistakeDetection/

In [ ]:
# Prima vai nella directory del repo
%cd /content/HiERO-main

# Poi esegui lo script
!python infer.py \
  --checkpoint /content/HiERO-main/checkpoints/hiero_egovlp.pth \
  --input /content/data/egovlp/egovlp/ \
  --output /content/outputs/embeddings \
  --num_clusters 8 \
  --device cuda

/content
2025-12-21 12:04:21.395134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766318661.587072    6190 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766318661.640731    6190 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766318662.055770    6190 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766318662.055807    6190 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766318662.055812    6190 computation_placer.cc:177] computation p

In [ ]:
# @title 🛠️ Fix: Recupero Cluster (8 steps) dai file singoli
import numpy as np
import glob
import os
from tqdm import tqdm

OUTPUT_DIR = "/content/outputs/embeddings"
NEW_ALL_EMB_PATH = os.path.join(OUTPUT_DIR, "all_embeddings_clusters.npz")

print(f"📂 Cerco file singoli in: {OUTPUT_DIR}")
npz_files = glob.glob(os.path.join(OUTPUT_DIR, "*_embeddings.npz"))
print(f"   Trovati {len(npz_files)} file.")

clusters_dict = {}
count_ok = 0

if len(npz_files) > 0:
    print("\n🔄 Estrazione chiave 'clusters' (8 steps)...")

    for fpath in tqdm(npz_files):
        try:
            filename = os.path.basename(fpath)
            # Il nome file è tipo "video_name.mp4_1s_1s_embeddings.npz"
            # Vogliamo la chiave pulita: "video_name.mp4_1s_1s"
            video_key = filename.replace("_embeddings.npz", "")

            data = np.load(fpath)

            # Cerchiamo varianti del nome chiave per sicurezza
            keys_in_file = list(data.files)
            target_key = None
            if 'clusters' in keys_in_file:
                target_key = 'clusters'
            elif 'embeddings' in keys_in_file and data['embeddings'].shape[0] == 8:
                 target_key = 'embeddings'

            if target_key:
                arr = data[target_key]
                if arr.shape[0] == 8: # Controllo extra dimensione
                    clusters_dict[video_key] = arr
                    count_ok += 1
                else:
                    pass # Dimensione errata

        except Exception as e:
            print(f"❌ Errore lettura {filename}: {e}")

    print(f"\n✅ Recuperati {count_ok} video con 8 cluster.")

    if count_ok > 0:
        # Salviamo il nuovo file aggregato
        np.savez_compressed(NEW_ALL_EMB_PATH, **clusters_dict)
        print(f"💾 Salvato nuovo file aggregato: {NEW_ALL_EMB_PATH}")

        # Analisi rapida del nuovo file
        sample_key = list(clusters_dict.keys())[0]
        sample_arr = clusters_dict[sample_key]
        print(f"\n🔬 Esempio '{sample_key}':")
        print(f"   Shape: {sample_arr.shape} (Dovrebbe essere 8x768)")
        print(f"   Dimensione: {sample_arr.shape[-1]} (Richiede proiezione -> 256)")
    else:
        print("⚠️ Nessun cluster trovato nei file singoli. Controlla i log di inferenza.")

else:
    print("❌ Nessun file .npz trovato nella cartella.")

📂 Cerco file singoli in: /content/outputs/embeddings
   Trovati 385 file.

🔄 Estrazione chiave 'clusters' (8 steps)...


100%|██████████| 385/385 [00:02<00:00, 169.37it/s]



✅ Recuperati 384 video con 8 cluster.
💾 Salvato nuovo file aggregato: /content/outputs/embeddings/all_embeddings_clusters.npz

🔬 Esempio '16_39_360p_224.mp4_1s_1s':
   Shape: (8, 768) (Dovrebbe essere 8x768)
   Dimensione: 768 (Richiede proiezione -> 256)


In [ ]:
# @title 4.6. 🔬 Analisi Finale Rigorosa: Verifica Struttura (384 video x 8 righe)
import numpy as np
import os

# Percorso del file clusters
CLUSTERS_PATH = "/content/outputs/embeddings/all_embeddings_clusters.npz"

print(f"📂 Analisi rigorosa file: {CLUSTERS_PATH} ...\n")

if not os.path.exists(CLUSTERS_PATH):
    print("❌ File non trovato!")
else:
    try:
        data = np.load(CLUSTERS_PATH, allow_pickle=True)
        keys = list(data.keys())
        total_videos = len(keys)

        print(f"1️⃣  CONTEGGIO VIDEO:")
        print(f"    Trovati {total_videos} video unici (chiavi).")

        # --- VERIFICA A TAPPETO SU TUTTI I VIDEO ---
        print(f"\n2️⃣  VERIFICA DIMENSIONI (Target: 8 righe x 768 colonne)...")

        correct_count = 0
        error_count = 0

        # Iteriamo su TUTTO il dataset
        for key in keys:
            arr = data[key]
            # Verifica rigida: deve essere (8, 768)
            if arr.shape == (8, 768):
                correct_count += 1
            else:
                error_count += 1
                # Stampiamo l'errore solo per i primi 3 casi per non intasare
                if error_count <= 3:
                    print(f"    ⚠️ Errore su '{key}': Ha shape {arr.shape} invece di (8, 768)")

        # --- REPORT FINALE ---
        print(f"\n📊 REPORT STATISTICO:")
        print(f"    ✅ Video corretti (8 step): {correct_count} su {total_videos}")
        print(f"    ❌ Video errati/incompleti: {error_count}")

        print(f"\n💡 CONCLUSIONE:")
        if error_count == 0 and total_videos > 0:
            print("    🎉 SUCCESSO TOTALE!")
            print("       Il file è perfettamente strutturato come un dizionario.")
            print("       Contiene esattamente 384 matrici, ognuna di 8 righe (step).")
            print("       Totale righe logiche: 3.072 (384 * 8).")
        else:
            print("    ⚠️ C'è qualche anomalia nei dati. Controlla gli errori sopra.")

    except Exception as e:
        print(f"❌ Errore critico lettura file: {e}")

📂 Analisi rigorosa file: /content/outputs/embeddings/all_embeddings_clusters.npz ...

1️⃣  CONTEGGIO VIDEO:
    Trovati 384 video unici (chiavi).

2️⃣  VERIFICA DIMENSIONI (Target: 8 righe x 768 colonne)...

📊 REPORT STATISTICO:
    ✅ Video corretti (8 step): 384 su 384
    ❌ Video errati/incompleti: 0

💡 CONCLUSIONE:
    🎉 SUCCESSO TOTALE!
       Il file è perfettamente strutturato come un dizionario.
       Contiene esattamente 384 matrici, ognuna di 8 righe (step).
       Totale righe logiche: 3.072 (384 * 8).


In [ ]:
# @title 5. 💾 Salvataggio Finale su Drive (File Unico + ZIP Backup con Progress Bar)
import os
import shutil
import zipfile
from google.colab import drive
from tqdm import tqdm

# Funzione per convertire byte in MB/GB
def format_bytes(size):
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'KB', 2: 'MB', 3: 'GB', 4: 'TB'}
    while size > power:
        size /= power
        n += 1
    return f"{size:.2f} {power_labels[n]}"

# Funzione per calcolare dimensione cartella
def get_dir_size(path):
    total = 0
    count = 0
    for entry in os.scandir(path):
        if entry.is_file():
            total += entry.stat().st_size
            count += 1
    return total, count

# 1. Mount Drive
if not os.path.exists('/content/drive'):
    print("📂 Montaggio Google Drive...")
    drive.mount('/content/drive')

# --- CONFIGURAZIONE ---
SOURCE_DIR = "/content/outputs/embeddings"
DEST_DIR = "/content/drive/MyDrive/MistakeDetection"

# File specifico da salvare separatamente
SOURCE_FILE_SINGLE = os.path.join(SOURCE_DIR, "all_embeddings_clusters.npz")
DEST_NAME_SINGLE = "hiero_all_video_steps.npz"
DEST_PATH_SINGLE = os.path.join(DEST_DIR, DEST_NAME_SINGLE)

# Nome per lo ZIP
ZIP_NAME = "hiero_video_steps.zip"
ZIP_PATH_LOCAL = os.path.join("/content", ZIP_NAME) # Salviamo prima in locale
ZIP_PATH_DRIVE = os.path.join(DEST_DIR, ZIP_NAME)
# ----------------------

if os.path.exists(SOURCE_DIR) and os.listdir(SOURCE_DIR):
    print("📊 ANALISI DIMENSIONI IN CORSO...")

    # Calcolo pesi
    single_size = os.path.getsize(SOURCE_FILE_SINGLE) if os.path.exists(SOURCE_FILE_SINGLE) else 0
    dir_size, file_count = get_dir_size(SOURCE_DIR)

    print(f"   🔹 File Unico (Clusters): {format_bytes(single_size)}")
    print(f"   🔹 Cartella Completa ({file_count} file): {format_bytes(dir_size)}")

    # Controllo Spazio Disco Colab
    total_disk, used_disk, free_disk = shutil.disk_usage("/")
    print(f"   💾 Spazio Disco Colab Libero: {format_bytes(free_disk)}")

    # Warning se lo spazio è stretto (consideriamo che lo zip occupa spazio + overhead)
    if free_disk < dir_size * 1.1:
        print("\n⚠️ ATTENZIONE CRITICA: Potrebbe non esserci abbastanza spazio su Colab per creare lo zip!")
        print("   Consiglio: Salva solo il file unico (A) e salta lo ZIP.")

    print("\n🚀 INIZIO OPERAZIONI...")

    # A) SALVATAGGIO FILE UNICO
    if os.path.exists(SOURCE_FILE_SINGLE):
        print(f"\n📄 [1/2] Copia file unico su Drive...")
        try:
            shutil.copy(SOURCE_FILE_SINGLE, DEST_PATH_SINGLE)
            print(f"   ✅ Salvato: {DEST_NAME_SINGLE} ({format_bytes(single_size)})")
        except Exception as e:
            print(f"   ❌ Errore copia: {e}")

    # B) CREAZIONE ZIP (MANUALE CON PROGRESS BAR)
    # Usiamo zipfile invece di shutil.make_archive per mostrare il progresso ed evitare timeout "ciechi"
    print(f"\n🗜️ [2/2] Creazione ZIP di backup ({format_bytes(dir_size)})...")
    try:
        file_list = [f for f in os.listdir(SOURCE_DIR) if os.path.isfile(os.path.join(SOURCE_DIR, f))]

        with zipfile.ZipFile(ZIP_PATH_LOCAL, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for f in tqdm(file_list, desc="Comprimendo", unit="file"):
                file_path = os.path.join(SOURCE_DIR, f)
                zipf.write(file_path, f) # Aggiunge al zip col nome 'f'

        zip_size = os.path.getsize(ZIP_PATH_LOCAL)
        print(f"   📦 Zip creato in locale: {format_bytes(zip_size)}")

        print(f"   ☁️ Caricamento su Drive in corso...")
        shutil.copy(ZIP_PATH_LOCAL, ZIP_PATH_DRIVE)
        print("   ✅ ZIP salvato su Drive!")

        # Pulizia spazio locale
        os.remove(ZIP_PATH_LOCAL)
        print("   🧹 File temporaneo locale rimosso.")

    except Exception as e:
        print(f"   ❌ Errore ZIP: {e}")

    print("\n🎉 PROCEDURA COMPLETATA!")

else:
    print("❌ Cartella output vuota.")

📊 ANALISI DIMENSIONI IN CORSO...
   🔹 File Unico (Clusters): 8.44 MB
   🔹 Cartella Completa (386 file): 17.35 GB
   💾 Spazio Disco Colab Libero: 46.60 GB

🚀 INIZIO OPERAZIONI...

📄 [1/2] Copia file unico su Drive...
   ✅ Salvato: hiero_all_video_steps.npz (8.44 MB)

🗜️ [2/2] Creazione ZIP di backup (17.35 GB)...


Comprimendo: 100%|██████████| 386/386 [17:16<00:00,  2.68s/file]


   📦 Zip creato in locale: 16.08 GB
   ☁️ Caricamento su Drive in corso...
   ✅ ZIP salvato su Drive!
   🧹 File temporaneo locale rimosso.

🎉 PROCEDURA COMPLETATA!
